# Species Richness by Location Map

This notebook generates and exports an HTML format interactive map of species richness at reported locations.

In [4]:
%run ../pathutils.ipynb
%run ../definitions.ipynb
%run ../database.ipynb
%run ../export.ipynb
%run pathutils.ipynb

In [5]:
import folium
from folium import CircleMarker
from folium.plugins import MarkerCluster
from pathlib import Path

def export_richness_map(export_folder_path, data, zoom):
    # Center map on the average latitude/longitude of all locations
    richness_map = folium.Map(
        location=[data["Latitude"].mean(), data["Longitude"].mean()],
        zoom_start=zoom,
        tiles="cartodbpositron"
    )

    # Cluster markers for many locations
    marker_cluster = MarkerCluster().add_to(richness_map)

    # Normalize marker size based on richness
    max_richness = data["Richness"].max()

    for _, row in data.iterrows():
        CircleMarker(
            location=(row["Latitude"], row["Longitude"]),
            radius=5 + 10 * (row["Richness"] / max_richness),  # Size scaled by richness
            color="blue",
            fill=True,
            fill_color="green",
            fill_opacity=0.6,
            popup=f"<b>{row['Location']}</b><br>Richness: {row['Richness']}",
        ).add_to(marker_cluster)

    # Export the map to HTML
    export_file_path = Path(export_folder_path) / "richness.html"
    richness_map.save(export_file_path.absolute())

In [ ]:
# Default zoom level for the map
zoom = 6

# Load the definitions
definitions = load_definitions("wildlife")
richness_definitions = [definition for definition in definitions if definition["Report"].casefold() == "richness"]

# Iterate over each definition to produce a report for each one
for definition in richness_definitions:
    for year in range(int(definition["From"]), 1 + int(definition["To"])):
        # Construct the query
        query = construct_query("wildlife", "location_richness_map.sql", {
            "COUNTRY": definition["Country"],
            "YEAR": str(year)
        })

        try:
            # Run the query to load the data - this raises a ValueError if there's no data
            df = query_data("wildlife", query)

            # Make sure the latitude and longitude have been read as numbers, not strings
            df["Latitude"] = pd.to_numeric(df["Latitude"], errors="coerce")
            df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")

            # Calculate the data for the richness chart
            richness = (
                df.groupby(["Location", "Latitude", "Longitude"])["Species"]
                .nunique()
                .reset_index()
            )

            richness.columns = ["Location", "Latitude", "Longitude", "Richness"]

            # Export the data to Excel
            export_folder_path = get_location_richness_folder_path(definition["Country"], year)
            export_to_spreadsheet(export_folder_path, "richness.xlsx", { "Location Richness": richness })

            # Export the interactive richness map
            export_richness_map(export_folder_path, richness, 6)

        except ValueError:
            print(f"WARNING: No data found for location = {definition['Location']}, category = {definition['Category']}")
